In [1]:
import json
import random
from datetime import datetime

In [2]:
f = open('./simulated_projects/projects.json', 'r')
data = json.load(f)

In [3]:
projects = data['projects']
users = data['users']
apps = data['app']
pms_systems = data['pms']

def get_users_by_domain(domain):
    return [user for user in users if user['domain'] == domain]

def get_apps_by_domain(domain):
    return [app for app in apps if domain in app['domain']]

def create_simulated_interaction(project, app, pms, user, process_instance_id, assigned_tasks):
    return {
        "timestamp": datetime.now().isoformat(),
        "process_instance_id": process_instance_id,
        "project_name": project['name'],
        "project_domain": project['domain'],
        "app_name": app['name'],
        "app_location": app['location'],
        "pms_name": pms['name'],
        "pms_location": pms['location'],
        "user_name": user['name'],
        "tasks": assigned_tasks
    }

In [4]:
simulated_data = []
total_connections_needed = 2000
min_connections_per_project = 1
max_connections_per_project = 2
process_instance_id_counter = 1

connections_counter = 0
stop_signal = False

used_combinations = set()

while not stop_signal:
    for project in projects:
        process_instance_id = process_instance_id_counter
        process_instance_id_counter += 1
        rand_nb_connections = random.randint(min_connections_per_project, max_connections_per_project)

        assigned_tasks = []
        if rand_nb_connections == 2:
            mid_point = random.choice(range(2, len(project["task"]) - 1))
            assigned_tasks.append(project["task"][:mid_point])
            assigned_tasks.append(project["task"][mid_point:])
        else:
            assigned_tasks = [project["task"]]

        for i in range(rand_nb_connections):
            compatible_apps = get_apps_by_domain(project['domain'])
            if not compatible_apps:
                continue
            
            app = random.choice(compatible_apps)
            pms = random.choice(pms_systems)
            available_users = get_users_by_domain(project['domain'])
            
            if not available_users:
                continue
            
            user = random.choice(available_users)
            while (process_instance_id, user['name']) in used_combinations:
                user = random.choice(available_users)

            interaction = create_simulated_interaction(project, app, pms, user, process_instance_id, assigned_tasks[i])
            simulated_data.append(interaction)
            used_combinations.add((process_instance_id, user['name']))

            connections_counter += 1
            if connections_counter == total_connections_needed:
                stop_signal = True
                break
        if stop_signal:
            break


In [5]:
# Save simulated data to a JSON file
with open('./simulated_projects/simulated_connections.json', 'w') as outfile:
    json.dump(simulated_data, outfile, indent=4)

print(f"Simulated data created with {len(simulated_data)} interactions.")

Simulated data created with 2000 interactions.
